In [1]:
#载入数据处理必备的模组
import numpy as np
import pandas as pd

In [2]:
#载入数据
data=pd.read_csv("data/data.csv")
print(data.head())
print(len(data))



  stock_code       revenue  s_fa_eps_basic     du_profit  plus_net_invest_inc  \
0  000001.SZ  8.196800e+10          1.0600  0.000000e+00         0.000000e+00   
1  000002.SZ  1.170000e+11          1.0050  2.222772e+10         2.046834e+09   
2  000008.SZ  1.007329e+09          0.0434  1.776341e+08         0.000000e+00   
3  000060.SZ  1.482919e+10          0.3800  1.279332e+09         2.853988e+07   
4  000063.SZ  4.775730e+10          0.9300  5.884933e+09        -1.951700e+07   

   net_cash_flows_oper_act  net_cash_flows_inv_act  net_roe  total_roe  \
0            -1.530000e+11           -9.716200e+10   8.7813     0.6289   
1             6.474047e+09            1.392036e+10   9.5305     1.6026   
2            -9.947078e+07           -1.193312e+09   1.9528     1.5235   
3             7.571307e+08           -4.932751e+08   8.6684     4.9238   
4            -3.170483e+09           -4.387188e+09  12.6504     2.9854   

         RSI       VOL      WILLR         y      date  
0  52.970207

In [3]:
#分析数据的日期
date=data.loc[:,"date"]
date=date.tolist()
date=set(date)
date=list(date)
date=sorted(date)
print(date)




[20171207, 20180105, 20180209, 20180309, 20180412, 20180510, 20180608, 20180706, 20180803, 20180831, 20181006, 20181103, 20181201, 20190102, 20190206, 20190306, 20190404, 20190507, 20190605, 20190703, 20190731, 20190829, 20191003, 20191031, 20191128, 20191227, 20200201]


In [4]:
#准备数据输入和标签
data=data.set_index(["stock_code"])


#数据清洗，去除NAN

data=data.dropna()

x_data=data.drop(["date","y"],axis=1)
y_data=data.loc[:,"y"]




In [5]:
#对于数据进行预处理
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

#对输入数据进行处理，提取可以被机器学习接受的数组形式

x=x_data.values

#对数据进行归一化处理
nm_scaler=MinMaxScaler()
x=nm_scaler.fit_transform(x)

# 使用主成分分析，对数据作降维处理，提取10个关键的特征

pca=PCA(n_components=7)
x=pca.fit_transform(x)



#划分训练和测试数据集

train_x=x[0:5702]
test_x=x[5702:]



In [6]:
#对标签数据进行处理
y=y_data.copy()
y=y.values

#把结果二值化用于分类的标签

y[y>=0.04]=1
y[y<0.04]=0


#划分训练和测试标签集

train_y=y[0:5702]
test_y=y[5702:]





In [7]:
# 构建最基础的机器学习KNN模型，对模型进行训练
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3,weights="distance")

knn.fit(train_x,train_y)
predict_train_y=knn.predict(train_x)




In [8]:
print(predict_train_y)

[0. 0. 1. ... 0. 0. 0.]


In [9]:
# 查看训练的拟合度

from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(train_y,predict_train_y))
cm=confusion_matrix(train_y,predict_train_y)
print(cm)


1.0
[[4263    0]
 [   0 1439]]


In [10]:
# 查看测试结果
predict_test_y=knn.predict(test_x)

print(accuracy_score(test_y,predict_test_y))
cm=confusion_matrix(test_y,predict_test_y)
print(cm)





0.5957642725598526
[[1132  226]
 [ 652  162]]


In [11]:
#搭建ANN模型，载入必须的库

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam


Using TensorFlow backend.
D:\Users\Anaconda3\envs\DL_interview\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\Anaconda3\envs\DL_interview\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\Anaconda3\envs\DL_interview\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\Anaconda3\envs\DL_interview\lib\sit

In [14]:
#搭建模型

model=Sequential([Dense(10,input_dim=7),Activation("tanh"),Dense(1),Activation("sigmoid")])

adam=Adam(lr=0.01)
model.compile(optimizer=adam,loss="mean_squared_error",metrics=["accuracy"])



In [15]:
#训练模型
model.fit(train_x,train_y,nb_epoch=100,batch_size=500)


Instructions for updating:
Use tf.cast instead.


D:\Users\Anaconda3\envs\DL_interview\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
5702/5702 [==============================] - 0s 30us/step - loss: 0.2231 - acc: 0.7099
Epoch 2/100
5702/5702 [==============================] - 0s 2us/step - loss: 0.1880 - acc: 0.7480
Epoch 3/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1843 - acc: 0.7478
Epoch 4/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1832 - acc: 0.7480
Epoch 5/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1823 - acc: 0.7489
Epoch 6/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1821 - acc: 0.7494
Epoch 7/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1818 - acc: 0.7501
Epoch 8/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1816 - acc: 0.7503
Epoch 9/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1816 - acc: 0.7510
Epoch 10/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1816 - acc: 0.7503
Epoch 11/100
5702/

5702/5702 [==============================] - 0s 3us/step - loss: 0.1800 - acc: 0.7499
Epoch 84/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1800 - acc: 0.7508
Epoch 85/100
5702/5702 [==============================] - 0s 2us/step - loss: 0.1800 - acc: 0.7503
Epoch 86/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1800 - acc: 0.7497
Epoch 87/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1800 - acc: 0.7504
Epoch 88/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1800 - acc: 0.7508
Epoch 89/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1800 - acc: 0.7511
Epoch 90/100
5702/5702 [==============================] - 0s 3us/step - loss: 0.1799 - acc: 0.7513
Epoch 91/100
5702/5702 [==============================] - 0s 4us/step - loss: 0.1799 - acc: 0.7508
Epoch 92/100
5702/5702 [==============================] - 0s 5us/step - loss: 0.1799 - acc: 0.7510
Epoch 93/100
5702/5702 

In [16]:
predict_test_y=model.predict(test_x)
print(predict_test_y)

[[0.19103503]
 [0.13108966]
 [0.19342291]
 ...
 [0.36240175]
 [0.2637457 ]
 [0.24705444]]


In [18]:
#测试模型表现
from sklearn.metrics import accuracy_score, confusion_matrix
#设定阈值
predict_test_y=model.predict(test_x)
theta=0.55
predict_test_y[predict_test_y>=theta]=1
predict_test_y[predict_test_y<theta]=0


#将量化预测结果进行阈值分类

acc=accuracy_score(test_y,predict_test_y)
cm=confusion_matrix(test_y,predict_test_y)

print(acc)
print(cm)


#查看统计结果


0.6270718232044199
[[1356    2]
 [ 808    6]]
